# pygmsh testing

In [ ]:
%matplotlib widget

In [ ]:
import gmsh
import numpy as np

In [ ]:
gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 1)
gmsh.option.setNumber("Mesh.Algorithm", 5) # delquad
gmsh.option.setNumber("Mesh.RecombineAll", 1)

gmsh.model.add("square")
gmsh.model.geo.addPoint(0, 0, 0, 1, 1)
gmsh.model.geo.addPoint(1, 0, 0, 1, 2)
gmsh.model.geo.addPoint(1, 1, 0, 1, 3)
gmsh.model.geo.addPoint(0, 1, 0, 1, 4)
gmsh.model.geo.addLine(1, 2, 1)
gmsh.model.geo.addLine(2, 3, 2)
gmsh.model.geo.addLine(3, 4, 3)

In [ ]:
# try automatic assignement of tag
line4 = gmsh.model.geo.addLine(4, 1)
gmsh.model.geo.addCurveLoop([1, 2, 3, line4], 1)
gmsh.model.geo.addPlaneSurface([1], 6)
gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)
#gmsh.write("square.unv")

In [ ]:
idx, x, _ = gmsh.model.mesh.get_nodes()

In [ ]:
x.reshape(-1,3)

In [ ]:
import k3d
import pygmsh

In [ ]:
X_Id = np.array([[0,0,0], [1,0,0], [1,1,0], [1, 0, 1]])
I_Fi = np.array([[0,1,2], [0,1,3]])
mesh_size = 2 * np.linalg.norm(X_Id[1]-X_Id[0])

X_Fid = X_Id[I_Fi]
with pygmsh.geo.Geometry() as geom:
    for X_id in X_Fid:
        geom.add_polygon(X_id, mesh_size=3)
    mesh = geom.generate_mesh()

X_Id_mesh = mesh.points.astype(np.float32)
I_Fi_mesh = mesh.cells[1][1].astype(np.uint32)

k3d_mesh = k3d.mesh(X_Id_mesh, I_Fi_mesh, side='double', color=0x75ade6)
k3d_mesh += k3d.mesh(X_Id_mesh, I_Fi_mesh, side='double', color=0x000000, wireframe=True)
k3d_mesh

In [ ]:
import gmsh
import numpy as np

In [ ]:
X_Id = np.array([[0,0,0], [1,0,0], [1,1,0], [1, 0, 1]])
I_Fi = np.array([[0,1,2], [0,1,3]])
mesh_size = 2 * np.linalg.norm(X_Id[1]-X_Id[0])

gmsh.initialize()
gmsh.clear()
gmsh.option.setNumber("General.Terminal", 1)
gmsh.option.setNumber("Mesh.Algorithm", 5) # delquad
gmsh.option.setNumber("Mesh.RecombineAll", 1)


X_Fid = X_Id[I_Fi]
X_tag = np.arange(len(X_Id))
points = np.array([
    gmsh.model.geo.addPoint(X_d[0], X_d[1], X_d[2], 1, tag)
    for tag, X_d in zip(X_tag, X_Id)
])
L_Fi = np.array([
    [gmsh.model.geo.addLine(I_i[0], I_i[1]),
     gmsh.model.geo.addLine(I_i[1], I_i[2]),
     gmsh.model.geo.addLine(I_i[2], I_i[0])]
    for I_i in I_Fi
])
loops = np.array([
    gmsh.model.geo.addCurveLoop([loop[0],loop[1],loop[2]])
    for loop in L_Fi
])
facets = np.array([
    gmsh.model.geo.addPlaneSurface([loop], tag)
    for tag, loop in enumerate(loops)
])

gmsh.model.geo.synchronize()
mesh = gmsh.model.mesh.generate(2)


In [ ]:
elem_types, elem_tags, node_tags = gmsh.model.mesh.get_elements()

In [ ]:
node_tags

In [ ]:
# extract cells
elem_types, elem_tags, node_tags = gmsh.model.mesh.getElements()
cells = []
for elem_type, node_tags in zip(elem_types, node_tags):
    # `elementName', `dim', `order', `numNodes', `localNodeCoord',
    # `numPrimaryNodes'
    num_nodes_per_cell = gmsh.model.mesh.getElementProperties(elem_type)[3]
    cells.append(node_tags.reshape(-1,3))
#     meshio.gmsh.gmsh_to_meshio_type
#     cells.append(
#         meshio.CellBlock(
#             meshio.gmsh.gmsh_to_meshio_type[elem_type],
#             np.asarray(node_tags).reshape(-1, num_nodes_per_cell) - 1,
#         )
#     )



In [ ]:
cells

In [ ]:
idx, x, _ = gmsh.model.mesh.get_nodes()

In [ ]:
gmsh.model.mesh.get_elements(2)

In [ ]:
x.reshape(-1,3)

**Note:** From the output of the last cell we see that pygmsh doesn't automatically merges the overlapping mesh (6 lines instead of 5)

## Obtaining lines mapping from facets mapping `I_Fi`

In [ ]:
I_Fi

In [ ]:
lines1 = I_Fi[:, [0, 1]]
lines2 = I_Fi[:, [0, 2]]
lines3 = I_Fi[:, [1, 2]]
lines = np.vstack((lines1, lines2, lines3))
lines

In [ ]:
lines = np.sort(lines,axis=1)
lines

In [ ]:
lines = np.unique(lines, axis=0)
lines

### All together

In [ ]:
def get_lines(I_Fi):
    lines1 = I_Fi[:, [0, 1]]
    lines2 = I_Fi[:, [0, 2]]
    lines3 = I_Fi[:, [1, 2]]
    lines = np.vstack((lines1, lines2, lines3))
    lines = np.sort(lines,axis=1)
    lines = np.unique(lines, axis=0)
    return lines

In [ ]:
import gmsh
import sys
import numpy as np

X_Id = np.array([[0,0,0], [1,0,0], [1,1,0], [1, 0, 1]])
I_Fi = np.array([[0,1,2], [0,1,3]])

gmsh.initialize()
gmsh.model.add("t1")

mesh_size = np.full(X_Id.shape[0], 0.1)

# TODO here, I'm taking only one surface, take all
points_coords = X_Id[I_Fi][0]

points = [gmsh.model.geo.add_point(*x, l) for x, l in zip(points_coords, mesh_size)]
curves = [gmsh.model.geo.add_line(points[k], points[k + 1]) for k in range(len(points) - 1)] + [gmsh.model.geo.add_line(points[-1], points[0])]

curve_loop = gmsh.model.geo.add_curve_loop(curves)
surface = gmsh.model.geo.add_plane_surface([curve_loop])

gmsh.model.geo.synchronize()

# Adding BCs
gmsh.model.add_physical_group(1, [1, 2], 555)
gmsh.model.set_physical_name(2, 555, "Fixed support")

gmsh.model.mesh.generate(3)

if '-nopopup' not in sys.argv:
    gmsh.fltk.run()

gmsh.finalize()

In [ ]:
# Old test
import gmsh
import sys

X_Id = np.array([[0,0,0], [1,0,0], [1,1,0]])
I_Fi = np.array([[0,1,2]])

gmsh.initialize()
gmsh.model.add("t1")

X_Fid = X_Id[I_Fi]
for point in X_Id:
    gmsh.model.geo.add_point(*point, 0.1)
    
# adding one because default points' tags start with 1
lines = get_lines(I_Fi) + 1
print(lines)
for line in lines:
    gmsh.model.geo.add_line(*line)

gmsh.model.geo.add_curve_loop([1, -2, 3])
gmsh.model.geo.add_plane_surface([1])
gmsh.model.geo.synchronize()

gmsh.model.mesh.generate(3)

if '-nopopup' not in sys.argv:
    gmsh.fltk.run()

gmsh.finalize()

## Test with the wb shell

In [ ]:
from bmcs_shell.folding.wb_shell_analysis import WBShellAnalysis
from bmcs_shell.folding.vmats2D_elastic import MATS2DElastic
# import bmcs_utils.api as bu
import pygmsh
import numpy as np

In [ ]:
tmodel = MATS2DElastic(E=28000, nu=0.2)
n_phi_plus=2
n_x_plus=1
wbm = WBShellAnalysis(F=-20000, h=10, tmodel=tmodel)
data = dict(alpha=1., a=500, a_high=3000, b=900, b_high=3000, c=400, c_high=2000,
            n_phi_plus=n_phi_plus, n_x_plus=n_x_plus, show_nodes=True)
wbm.geo.trait_set(**data);

In [ ]:
wbm.F = -700
wbm.show_wireframe = True
wbm.xdomain.mesh.direct_mesh = False
wbm.xdomain.mesh.subdivision = 2

In [ ]:
wbm.app()

In [ ]:
wbm.xdomain.mesh.direct_mesh = False

In [ ]:
X_Ia = wbm.geo.X_Ia  # nodal coordinates
X_Ia, len(X_Ia)

In [ ]:
I_Fi = wbm.geo.I_Fi  # elements

In [ ]:
wbm.xdomain.mesh.X_Id.shape

In [ ]:
X_Fid = X_Ia[I_Fi]

In [ ]:
import numpy as np

U_1 = wbm.hist.U_t[-1]
# U_max = np.max(np.fabs(U_1))
# U_max
U_1 
U_1.reshape(-1, 3) * 1
# _, loaded_nodes, _ = wbm.bc_loaded
# wbm.xdomain.I_Ei.astype(np.uint32)
bc_fixed, fixed_nodes, fixed_dofs = wbm.bc_fixed
bc_fixed[5]

U_1 = wbm.hist.U_t[-1]
np.max(np.fabs(U_1))